In [1]:
using DynamicalSystems, Plots, NPZ
import Distributions: Distribution, Normal
include("../src/KoopmanCausality.jl")
using .KoopmanCausality

In [2]:
X2_train = npzread("rossler_confounder_data/rossler_confounder_train_X2.npy")
X3_train = npzread("rossler_confounder_data/rossler_confounder_train_X3.npy")
Y2_train = npzread("rossler_confounder_data/rossler_confounder_train_Y2.npy")
Y3_train = npzread("rossler_confounder_data/rossler_confounder_train_Y3.npy")
size(X2_train)

(3, 500001)

In [3]:
X2_test = npzread("rossler_confounder_data/rossler_confounder_test_X2.npy")
X3_test = npzread("rossler_confounder_data/rossler_confounder_test_X3.npy")
Y2_test = npzread("rossler_confounder_data/rossler_confounder_test_Y2.npy")
Y3_test = npzread("rossler_confounder_data/rossler_confounder_test_Y3.npy")
size(X2_test)

(3, 20001, 100)

hyperparameters for brute-force optimization 

In [6]:
σs = [2.0, 1.0, 0.5, 0.25, 0.1, 0.05, 0.01, 0.005, 0.001, 0.0005]
N_samples = 5
N_features = 150
N_vars_marg = 3
N_vars_joint = 6;

### 2-> 3

### setup the model space for hyperparameter optimization

Randomly sample RFFs 5 times for each value of the variance hyperparameter $\sigma$ and create marginal and joint models for each of these dictionaries. 

In [7]:
expanded_σs = []
marg_models = []
joint_models = []
for σ in σs
    for _ in 1:N_samples
        push!(expanded_σs, σ)
        marg_dist = Normal(0.0, σ)
        marg_dict = RFF_dict(N_features, N_vars_marg, marg_dist)
        marg_model = causal_DMD(X3_train, Y3_train, marg_dict)
        push!(marg_models, marg_model)
        
        joint_dist = Normal(0.0, σ)
        joint_dict = RFF_dict(N_features, N_vars_joint, joint_dist)
        joint_model = causal_DMD(X3_train, X2_train, Y3_train, joint_dict)
        push!(joint_models, joint_model)
    end
end

For each test case, evaluate all of the models in the model space and take those with minimal error to compute the Koopman causality measure

In [8]:
N_tests = size(X3_test, 3)
koop_causes = Vector{Float64}(undef, N_tests)
for n in 1:N_tests
    marg_err = Inf
    joint_err = Inf
    for(marg_model, joint_model) in zip(marg_models, joint_models)
        marg = causal_eval(X3_test[:,:, n], Y3_test[:,:, n], marg_model)
        marg_err = min(marg_err, marg)
        joint = causal_eval(X3_test[:,:, n], X2_test[:,:, n], Y3_test[:,:, n], joint_model)
        joint_err = min(joint_err, joint)
    end
    kcause = marg_err - joint_err
    koop_causes[n] = kcause
end

In [9]:
maximum(koop_causes)

0.03259657827503104

In [10]:
minimum(koop_causes)

-0.08463581607284398

In [11]:
count = 0
for kc in koop_causes
    if kc < 0
        count += 1
    end
end
count

90

No causality is correctly identified in 90% of test cases, the other 10% incorrectly identify causality from the spurious correlation

### 3-> 2

### setup the model space for hyperparameter optimization

Randomly sample RFFs 5 times for each value of the variance hyperparameter $\sigma$ and create marginal and joint models for each of these dictionaries. 

In [12]:
expanded_σs = []
marg_models = []
joint_models = []
for σ in σs
    for _ in 1:N_samples
        push!(expanded_σs, σ)
        marg_dist = Normal(0.0, σ)
        marg_dict = RFF_dict(N_features, N_vars_marg, marg_dist)
        marg_model = causal_DMD(X2_train, Y2_train, marg_dict)
        push!(marg_models, marg_model)
        
        joint_dist = Normal(0.0, σ)
        joint_dict = RFF_dict(N_features, N_vars_joint, joint_dist)
        joint_model = causal_DMD(X2_train, X3_train, Y2_train, joint_dict)
        push!(joint_models, joint_model)
    end
end

For each test case, evaluate all of the models in the model space and take those with minimal error to compute the Koopman causality measure

In [13]:
N_tests = size(X3_test, 3)
koop_causes = Vector{Float64}(undef, N_tests)
for n in 1:N_tests
    marg_err = Inf
    joint_err = Inf
    for(marg_model, joint_model) in zip(marg_models, joint_models)
        marg = causal_eval(X2_test[:,:, n], Y2_test[:,:, n], marg_model)
        marg_err = min(marg_err, marg)
        joint = causal_eval(X2_test[:,:, n], X3_test[:,:, n], Y2_test[:,:, n], joint_model)
        joint_err = min(joint_err, joint)
    end
    kcause = marg_err - joint_err
    koop_causes[n] = kcause
end

In [14]:
maximum(koop_causes)

0.008715766097400213

In [15]:
minimum(koop_causes)

-0.1266983718977024

In [16]:
count = 0
for kc in koop_causes
    if kc < 0
        count += 1
    end
end
count

97

No causality is correctly identified in 97% of test cases, the other 3% incorrectly identify causality from the spurious correlation